### Multivariate Linear Regression
#### Published 2010, 2012. 


In [28]:
# Imports
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd


In [34]:
#Load in data sets

#outline_points = np.loadtxt('H:\Personal\AVOA\AVOA_crunch_Alpha_1.0\Python_reWrite\survey_corners', dtype='int')
outline_points = np.loadtxt(r'C:\Users\Duxvader\dux_python\fracture_detection\survey_corners', dtype='int')
trc_data = np.loadtxt(r'C:\Users\Duxvader\dux_python\fracture_detection\trc_data_py', dtype='int')
amp_data = np.loadtxt(r'C:\Users\Duxvader\dux_python\fracture_detection\amp_data_cap_py')
#print(outline_points)
#plt.plot(outline_points[:,0], outline_points[:,1])
#plt.show(); #Visual QC

In [37]:
#print(trc_data)